# intro loads

## load torchvision & ivy models 

In [1]:
import torch
import torch.nn.functional as F
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
from torchvision import transforms

import ivy
import json
import os
from utils import download_weights
from efficientnetv1 import EfficientNetV1

ivy.set_torch_backend()

# Load torchvision model
torch_model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
torch_model.eval()  # Set the model to evaluation mode

# Load Ivy model
device = "cpu"
with open("variant_configs.json") as json_file:
    configs = json.load(json_file)
configs = configs["v1"]
base_model = configs["base_args"]
phi_values = configs["phi_values"]["b0"]
def create_model():
    ivy_model = EfficientNetV1(
        base_model, phi_values, 1000, device=device, training=False
    )

    # copy weights
    weight_path = "weights/b0.pickled"
    if not os.path.isfile(weight_path):
        download_weights(weight_path)

    torch_v = ivy.Container.cont_from_disk_as_pickled(weight_path)
    torch_list_weights = torch_v.cont_to_flat_list()
    assert len(ivy_model.v.cont_to_flat_list()) == len(torch_list_weights)

    ivy_model.v.classifier.submodules.v1.b = ivy.Array(
        torch_list_weights[0].detach().cpu().numpy()
    ).to_device(device)
    ivy_model.v.classifier.submodules.v1.w = ivy.Array(
        torch_list_weights[1].detach().cpu().numpy()
    ).to_device(device)
    del torch_list_weights[:2]

    def _copy_weights(dictionary):
        for key, value in list(dictionary.items()):
            # print(key, type(value))
            if isinstance(value, dict):
                _copy_weights(value)
            else:
                assert (
                    torch_list_weights[0].shape == value.shape
                ), f"{torch_list_weights[0].shape}, {value.shape}"
                dictionary.pop(key)
                dictionary[key] = ivy.Array(
                    torch_list_weights[0].detach().cpu().numpy()
                ).to_device(device)
                del torch_list_weights[0]
    for k, v in ivy_model.v.features.submodules.items():
        _copy_weights(v)
    assert len(torch_list_weights) == 0
    return ivy_model

ivy_model = create_model()

2023-05-21 20:31:20.671716: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 20:31:21.196860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## load image

In [2]:
# image_path = "images/ILSVRC2012_test_00000007.jpeg"
image_path = 'images/ILSVRC2012_test_00000030.jpeg'
# Define the transformation pipeline
preprocess = transforms.Compose(
    [
        transforms.Resize(
            256, interpolation=transforms.InterpolationMode.BICUBIC
        ),  # Resize the image to a square of size 256x256
        transforms.CenterCrop(224),  # Crop the center portion of the image to 224x224
        transforms.ToTensor(),  # Convert the PIL image to a tensor
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # Normalize the image
    ]
)

image = Image.open(image_path)
input_tensor = preprocess(image).unsqueeze(0)  # Add a batch dimension
numpy_image = input_tensor.detach().cpu().numpy()
numpy_image = numpy_image.reshape((1, 224, 224, 3))

ivy_image = ivy.Array(numpy_image).to_device(device)

# compare

In [3]:
output = ivy.softmax(ivy_model(ivy_image))
output[0].argmax(), sorted(output[0])[-5:]

(ivy.array(902),
 [ivy.array(0.02553806),
  ivy.array(0.03418851),
  ivy.array(0.06302106),
  ivy.array(0.08793806),
  ivy.array(0.13497415)])

In [6]:
with torch.no_grad():
    output = F.softmax(torch_model(input_tensor), dim=1)
output[0].argmax(), sorted(output[0])[-5:]

(tensor(97),
 [tensor(0.0007),
  tensor(0.0007),
  tensor(0.0024),
  tensor(0.0029),
  tensor(0.9771)])

# ablation study?

In [7]:
ivy_model.features._submodules[0].conv._submodules[0](ivy_image).reshape(shape=(1, 32, 112, 112))[0][0][0][:5]

ivy.array([-0.14932327, -0.89016241,  1.44651437, -0.38861609, -1.47516894])

In [9]:
# if equal proves both models are receiving the same input
torch_model.features[0][0](input_tensor)[0][0][0][:5] == torch_model.features[0][0](torch.Tensor(numpy_image).reshape(1, 3, 224, 224))[0][0][0][:5]

tensor([True, True, True, True, True])

In [10]:
torch_model.features[0][0](input_tensor)[0][0][0][:5]

tensor([-5.9799, -6.0744, -5.7570, -5.8083, -5.2820], grad_fn=<SliceBackward0>)